In [ ]:
%matplotlib inline
import PIL
import tqdm
import pickle
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from skimage.filters import gabor
from skimage.feature import local_binary_pattern, greycomatrix, greycoprops

In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
import shutil
from sklearn.model_selection import train_test_split
import os
import shutil

import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
import random
import pickle
from sklearn import model_selection
from pylab import *
#from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import scipy.fftpack
from skimage import data, io
from scipy.signal import convolve2d as conv2
#import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import train_test_split,cross_val_score
from skimage import color, data, restoration
from skimage.feature import greycomatrix,greycoprops

from sklearn.metrics import  accuracy_score, f1_score, precision_score,confusion_matrix, recall_score, roc_auc_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC

%matplotlib inline
from sklearn import preprocessing

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
import skimage.io as io
import skimage as sk
from skimage import measure
import math
from math import sqrt
from skimage import io, color, img_as_ubyte
from skimage.feature import greycomatrix, greycoprops
from sklearn.metrics.cluster import entropy
from scipy.stats import skew, kurtosis
from numpy import asarray
from PIL import Image

import math
from math import sqrt

import skimage as sk
from skimage import measure

from numpy import asarray
from PIL import Image
import math
from math import sqrt

import skimage as sk
from skimage import measure

from numpy import asarray
from PIL import Image

import numpy as np
from skimage import io, color, img_as_ubyte
from skimage.feature import greycomatrix, greycoprops
from sklearn.metrics.cluster import entropy

import scipy
import cv2
from scipy.stats import skew, kurtosis
import skimage
from skimage.filters import threshold_otsu

In [ ]:
!unzip -u "/content/drive/MyDrive/Research HIRL/Breast Ultrasound/Datasets/Malignant ROI.zip" -d "/content"

print('\n\ndone!')

In [ ]:
####################### feature extraction###################
#############give directory
dirr1= '/content/drive/MyDrive/Research HIRL/Breast Ultrasound/Datasets/Breast Ultrasound/Only Breast US/Mal' #### source folder

csv='/content/mal raw.csv' ### csv save path
### must change csv path
c=1 ### c=class 1=0 or class 2=1

#####################################################################################################
black=0
j=0
df1 = pd.DataFrame(columns=['class', 'Area', 'C_ratio','ecl_ratio', 'Solidity', 'Circularity', 'EquivDiameter', 'ConvexArea', 'Extent', 'FilledArea', 'Major axis length', 'Minor axis length', 'Mean', 'Standard Deviation', 'Shannon_Entropy', 'Skewness', 'kurtosis','lbp_energy','lbp_entropy','gabor_energy','gabor_entropy'])

import os
onlyfiles = next(os.walk(dirr1))[2] #dir is your directory path as string
print(len(onlyfiles))


for i in os.listdir(dirr1):
    path = os.path.join(dirr1,i)
    im = io.imread(path)
    height = im.shape[0]
    width = im.shape[1]

    print('working on img ',i)
    props=sk.measure.regionprops(im)
    if len(props)!=0:
        pr=[]
        for prop in props:
            pr.append(prop.area)
        num = np.array(pr)
        a=props[np.argmax(num)]

        imgcv = cv2.imread(path)
        canny = cv2.Canny(imgcv, 100, 200)
        points = np.argwhere(canny>0)
        center, radius = cv2.minEnclosingCircle(points)
        # draw circle on copy of input
        result = imgcv.copy()
        x2 = int(center[1])
        y2 = int(center[0])
        rad2 = int(radius)
        cv2.circle(result, (x2,y2), rad2, (255,255,255), 1)


        gray = cv2.cvtColor(imgcv,cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        ave = cv2.mean(thresh)[0]/255
        area = ave*height*width###########################
        ccarea = math.pi * (radius ** 2)
        circle_ratio = ccarea/area
        try:

            ecl_cnt = max(contours, key = cv2.contourArea)
            ecl_area = cv2.contourArea(ecl_cnt)
            ecl_ratio = area / ecl_area
        except:
            ecl_ratio = 0

        try:
            image=skimage.io.imread(path,as_gray=True)
            thresh = threshold_otsu(image)
            binary = image > thresh
            perimeter=measure.perimeter(binary)################

            PA_ratio=round(perimeter/area,4)
            EquivDiameter=round(sqrt(4*area/math.pi),4)
            Circularity=round((4*area*math.pi)/(perimeter*perimeter),4)
            ConvexArea=round(a.convex_area,4)

        except:

            print('black found \n')
            black+=1
            continue
        Extent=round(a.extent,4)
        Solidity=round(area/ConvexArea,4)
        FilledArea=round(a.filled_area,4)
        maj_ax_le=round(a.major_axis_length,4)
        min_ax_le=round(a.minor_axis_length,4)
        #LS_ratio=round(maj_ax_le/min_ax_le,4)
        EulerNumber=round(a.euler_number,4)

        image=Image.open(path)
        pixels=asarray(image)
        # convert from integers to floats
        pixels=pixels.astype('float32')
        # calculate global mean
        mean=round(pixels.mean(),3)

        pixels=asarray(image)
        # convert from integers to floats
        pixels=pixels.astype('float32')
        # calculate global mean and standard deviation
        std=round(pixels.std(),3)

        #print('Standard Deviation: %.3f' % (std))
        sha_entropy=round(skimage.measure.shannon_entropy(image),4)
        #LBP & Gabor
        img_gray = image.convert('L')
        img_arr = np.array(img_gray)
        feat_lbp = local_binary_pattern(img_arr,8,1,'uniform') #Radius = 1, No. of neighbours = 8
        feat_lbp = np.uint8((feat_lbp/feat_lbp.max())*255)
        lbp_hist,_ = np.histogram(feat_lbp,8)
        lbp_hist = np.array(lbp_hist,dtype=float)
        lbp_prob = np.divide(lbp_hist,np.sum(lbp_hist))
        lbp_energy = np.sum(lbp_prob**2)
        lbp_entropy = -np.sum(np.multiply(lbp_prob,np.log2(lbp_prob)))

        gaborFilt_real,gaborFilt_imag = gabor(img_arr,frequency=0.6)
        gaborFilt = (gaborFilt_real**2+gaborFilt_imag**2)//2
        gabor_hist,_ = np.histogram(gaborFilt,8)
        gabor_hist = np.array(gabor_hist,dtype=float)
        gabor_prob = np.divide(gabor_hist,np.sum(gabor_hist))
        gabor_energy = np.sum(gabor_prob**2)
        gabor_entropy = -np.sum(np.multiply(gabor_prob,np.log2(gabor_prob)))

        rgbImg=io.imread(path)
        #grayImg=img_as_ubyte(color.rgb2gray(rgbImg))
        distances=[1,2,3]
        angles=[0,np.pi/4,np.pi/2,3*np.pi/4]
        properties=['energy','homogeneity']
        #glcm=greycomatrix(rgbImg,distances=distances,angles=angles,symmetric=True,normed=True)
        #glcm_entropy=round(-np.sum(glcm*np.log2(glcm+(glcm==0))),4)
        im=cv2.imread(path)
        colon_number=0
        Skewness=round(skew(im.reshape(-1,3)[:,colon_number]),4)
        Kurtosis=round(kurtosis(im.reshape(-1,3)[:,colon_number]),4)
        #df1.loc[j] =list(area)+list(PA_ratio)+list(Solidity)+list(Circularity)+list(EquivDiameter)+list(ConvexArea)+list(Extent)+list(FilledArea)+list(maj_ax_le)+list(min_ax_le)+list(mean)+list(std)+list(sha_entropy)+list(glcm_entropy)+list(Skewness)+list(Kurtosis)
        df1.loc[j]=({'class':c,'Area':area,
                          'C_ratio': circle_ratio,
                          'ecl_ratio':ecl_ratio,
                          'PA_ratio':PA_ratio,
                          'Solidity':Solidity,
                          'Circularity':Circularity,
                          'EquivDiameter':EquivDiameter,
                          'ConvexArea':ConvexArea,
                          'Extent':Extent,
                          'FilledArea':FilledArea,
                          'Major axis length':maj_ax_le,
                          'Minor axis length':min_ax_le,
                          #'LS_ Ratio':[LS_ratio],
                          'Mean':mean,
                          'Standard Deviation':std,
                          'Shannon_Entropy':sha_entropy,
                          #'Glcm_entropy':glcm_entropy,
                          'Skewness':Skewness,
                          'kurtosis':Kurtosis,
                          'lbp_energy':lbp_energy,
                          'lbp_entropy':lbp_entropy,
                          'gabor_energy':gabor_energy,
                          'gabor_entropy':gabor_entropy})

        j+=1
    else:
        print('black found \n')
        black+=1
    print('\n')

print('\n\n\n')
print('black images found: ',black, 'out of total image : ',len(onlyfiles), )
print('\n\n\n\n\n\n')

frames = df1
############################################### give directy to save csv file
#roidata_set = pd.concat(frames)
frames.to_csv(csv,index=False)### give train.csv path
#print(frames)
###############################################################################
######## after completing train set give me an ss of the output that will be printed below... do the same for val and test also.

NO need


In [ ]:
dirr1= '/content/drive/MyDrive/Research HIRL/Breast Ultrasound/Datasets/Breast Ultrasound/Only Breast US/Ben' #### source folder
 #### source folder

csv= "/content/beni.csv"
 ### csv save path
### must change csv path

c= 0 ### c=class 1=0 or class 2=1

#####################################################################################################

j=0
df1 = pd.DataFrame(columns=['class', 'Solidity', 'sift_keypoints', 'wh_ratio', 'ecl_ratio', 'harris_corner', 'circularity'])
#df1 = pd.DataFrame(columns=['class', 'wb_ratio'])
import os
onlyfiles = next(os.walk(dirr1))[2] #dir is your directory path as string
#print(len(onlyfiles))

for i in os.listdir(dirr1):
    path = os.path.join(dirr1,i)
    img = cv2.imread(path)
    print('working on img ',i)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)




# solidity, perimeter,circularity
    ret,thresh = cv2.threshold(gray,150,255,0)
    contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key = cv2.contourArea)
    area = cv2.contourArea(cnt)
    hull = cv2.convexHull(cnt)
    hull_area = cv2.contourArea(hull)
    Solidity = float(area)/hull_area
    perimeter = cv2.arcLength(cnt, True)
    circularity=((4*3.1416*area)/(perimeter**2))

# find the SIFT extreme keypoints on image (grayscale)
    sift = cv2.SIFT_create()
    kp = sift.detect(gray,None)
    sift_keypoints = (len(kp))



# find the wh_ratio
    x,y,w,h = cv2.boundingRect(cnt)
    wh_ratio = w/h



 # find the ecl_ratio
    M = cv2.moments(thresh)
    #calculate x,y coordinate of center
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    center_coordinates = (cX, cY)
    x,y,w,h = cv2.boundingRect(cnt)
    #x,y,w,h,a = cv2.minAreaRect(cnt)
    w = int(w/2)
    h = int(h/2)
    axesLength = (w,h)
    angle = 0
    startAngle = 0
    endAngle = 360
    color = (0, 0, 255)
    thickness = -1
    eclipse = cv2.ellipse(img, center_coordinates, axesLength, angle, startAngle, endAngle, color, thickness)
    lower_blue = np.array([0, 0, 222])
    upper_blue = np.array([0, 0, 255])
    mask_blue = cv2.inRange(eclipse, lower_blue, upper_blue)
    contours,hierarchy = cv2.findContours(mask_blue, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    try:
        ecl_cnt = max(contours, key = cv2.contourArea)
        ecl_area = cv2.contourArea(ecl_cnt)
        ecl_ratio = area / ecl_area
    except:
        ecl_ratio = 0



#harris corner
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray,5,3,0.04)
    ret, dst = cv2.threshold(dst,0.1*dst.max(),255,0)
    dst = np.uint8(dst)
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)
    harris_corner = len(corners)

#wb ratio

    df1.loc[j]=({'class':c,'Solidity':Solidity, 'sift_keypoints' : sift_keypoints, 'wh_ratio' : wh_ratio, 'ecl_ratio' : ecl_ratio,
                 'harris_corner' : harris_corner, 'circularity' : circularity})
    j+=1

frames = df1


############################################### give directy to save csv file
frames.to_csv(csv,index=False)### give train.csv path

###############################################################################

working on img  benign (6).png
working on img  benign (265).png
working on img  benign (82).png
working on img  benign (13).png
working on img  benign (55).png
working on img  benign (29).png
working on img  benign (199).png
working on img  benign (387).png
working on img  benign (107).png
working on img  benign (416).png
working on img  benign (116).png
working on img  benign (194).png
working on img  benign (235).png
working on img  benign (318).png
working on img  benign (23).png
working on img  benign (397).png
working on img  benign (214).png
working on img  benign (21).png
working on img  benign (237).png
working on img  benign (66).png
working on img  benign (289).png
working on img  benign (100).png
working on img  benign (169).png
working on img  benign (345).png
working on img  benign (331).png
working on img  benign (205).png
working on img  benign (247).png
working on img  benign (3).png
working on img  benign (142).png
working on img  benign (321).png
working on img  benig

In [ ]:
import pandas as pd
from scipy.stats import ttest_ind

# Create example dataframe
df_0 = pd.read_csv('/content/beni.csv')
df_1 = pd.read_csv('/content/malig.csv')


In [ ]:
df_0['class'] = 0

In [ ]:
df4 = pd.concat([df_0, df_1], axis=0)
df4

,class,Solidity,sift_keypoints,wh_ratio,ecl_ratio,harris_corner,circularity
0,0,0.196905,1008,1.202765,0.154532,23,0.024871
1,0,0.665531,1306,2.717172,0.657507,32,0.092231
2,0,0.584761,598,3.347222,0.406877,15,0.073469
3,0,0.500723,407,5.106061,0.467705,9,0.069544
4,0,0.483891,1267,3.650718,0.422755,77,0.031343
...,...,...,...,...,...,...,...
205,1,0.373745,1404,4.025641,0.275300,18,0.041558
206,1,0.708790,829,2.292683,0.845348,3,0.073309
207,1,0.322469,431,7.133333,0.294987,2,0.019893
208,1,0.628049,762,7.833333,0.457778,2,0.129956


In [ ]:
import pandas as pd
from scipy.stats import ttest_ind


# Split dataframe into two based on target class
df_0 = df4[df4["class"] == 0]
df_1 = df4[df4["class"] == 1]

# Perform t-tests on each predictor column and save results in new dataframe
results_df = pd.DataFrame(columns=["Variable", "T-Statistic", "P-Value"])
for col in df4.columns[:-1]:
    t_stat, p_val = ttest_ind(df_0[col], df_1[col])
    results_df = results_df.append({"Variable": col, "T-Statistic": t_stat, "P-Value": p_val}, ignore_index=True)

# Print results dataframe
print(results_df)

# Save results dataframe to CSV file
results_df.to_csv("t_test_results_surface_area.csv", index=False)


         Variable  T-Statistic       P-Value
0           class         -inf  0.000000e+00
1        Solidity    -2.655503  8.114644e-03
2  sift_keypoints     6.929755  1.023882e-11
3        wh_ratio     0.019135  9.847393e-01
4       ecl_ratio    -2.803666  5.204471e-03
5   harris_corner     1.369388  1.713545e-01


In [ ]:
df4

In [ ]:
import pandas as pd

# Assuming you have a DataFrame named 'combined_df' containing both classes

# List of specific features you want to select
selected_features = ['sift_keypoints', 'ecl_ratio', 'class']

# Create a new DataFrame with only the selected features
new_df1 = df4[selected_features].copy()
new_df1
# Now, 'new_df' contains only the selected features from 'combined_df'


,sift_keypoints,ecl_ratio,class
0,1008,0.154532,0
1,1306,0.657507,0
2,598,0.406877,0
3,407,0.467705,0
4,1267,0.422755,0
...,...,...,...
205,1404,0.275300,1
206,829,0.845348,1
207,431,0.294987,1
208,762,0.457778,1


In [ ]:
new_df1

,sift_keypoints,ecl_ratio,class
0,1008,0.154532,0
1,1306,0.657507,0
2,598,0.406877,0
3,407,0.467705,0
4,1267,0.422755,0
...,...,...,...
205,1404,0.275300,1
206,829,0.845348,1
207,431,0.294987,1
208,762,0.457778,1


In [ ]:
df5 = pd.concat([new_df, new_df1], axis=1)
df5

,PA_ratio,Solidity,Circularity,Mean,lbp_energy,lbp_entropy,gabor_energy,sift_keypoints,ecl_ratio,class
0,0.0438,0.5240,0.0252,63.405,0.152097,2.849728,0.561409,1008,0.154532,0
1,0.0861,0.3746,0.0095,98.725,0.177897,2.749387,0.315603,1306,0.657507,0
2,0.0334,0.3401,0.0381,57.262,0.166834,2.767224,0.631810,598,0.406877,0
3,0.0469,1.0306,0.0157,67.442,0.178055,2.715836,0.545905,407,0.467705,0
4,0.0381,1.0285,0.0186,97.774,0.197450,2.643582,0.289209,1267,0.422755,0
...,...,...,...,...,...,...,...,...,...,...
205,0.0766,0.3317,0.0060,71.673,0.181620,2.721959,0.561485,1404,0.275300,1
206,0.0251,0.9446,0.0767,103.236,0.162167,2.810447,0.305326,829,0.845348,1
207,0.0176,0.3527,0.1339,50.123,0.158736,2.807216,0.584444,431,0.294987,1
208,0.0530,0.6066,0.0174,43.219,0.163180,2.788491,0.790540,762,0.457778,1


In [ ]:
df5.to_csv('raw_us.csv')